In [23]:
import os
import openai
import pandas as pd
import tiktoken
import json
import requests
import pickle as pkl
from collections import defaultdict
from googlesearch import search
from fake_useragent import UserAgent
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import re
from time import sleep

In [2]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

In [3]:
print(deployment_name)

ADTech2


In [4]:
#To check if it's working

import os
import openai
openai.api_type = "azure"
openai.api_version = "2023-05-15" 

print(deployment_name)

response = openai.ChatCompletion.create(
    engine=deployment_name, # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

print(response)

print(response['choices'][0]['message']['content'])

ADTech2
{
  "id": "chatcmpl-805luCyj03hgWS1amtPvpHoVngC4f",
  "object": "chat.completion",
  "created": 1695032878,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Microsoft was founded by Bill Gates and Paul Allen in April 1975."
      }
    }
  ],
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 31,
    "total_tokens": 46
  }
}
Microsoft was founded by Bill Gates and Paul Allen in April 1975.


In [5]:
system_message={"role": "system", "content": "You are a helpful assistant."}

In [6]:
max_response_tokens = 500
token_limit = 4096
conversation = []
conversation.append(system_message)

In [7]:
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        # print(message)
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [8]:
df_IAB=pd.read_csv('../data/IAB_Evaluation.csv')

In [16]:
Tiers=df_IAB['IAB_Label'].to_list()


In [15]:
def prompt(adword):

    prompt="""

For the following adword: {} 
Try to give 5 synonyms that are shorter in length. Return the results in a bulleted format.

""".format(adword)

    return prompt

In [17]:
def prompt_output_parse(output):
    first_split=output.split('\n')
    synonyms=[]
    for items in first_split:
        synonyms.append(items.split('-')[-1].strip())

    return synonyms

In [18]:
def deployment_stuff():
    with open('azure-configuration.json') as inputfile:
        azureconfig = json.load(inputfile)
    openai.api_key = azureconfig['key'] 
    openai.api_base = azureconfig['endpoint'] 
    openai.api_type = 'azure'
    openai.api_version = '2023-05-15' # this may change in the future

    deployment_name= azureconfig['deployment_name']

    return deployment_name

    

In [20]:
def call(deployment_name, adword):
    #Right now calling requests to get the synopsis, once Dataset is ready, replace it
    #context=get_synopsis(series, episode, context)

    #OpenAI Stuff
    system_message={"role": "system", "content": "You are a helpful assistant."}
    max_response_tokens = 4096
    token_limit = 2000
    conversation = []
    conversation.append(system_message)


    user_input = prompt(adword)

    print(f"Adword was {adword}")
    
    conversation.append({"role": "user", "content": user_input})
    #conv_history_tokens = num_tokens_from_messages(conversation)
    # print(conv_history_tokens)

    # while conv_history_tokens + max_response_tokens >= token_limit:
    #     del conversation[0] 
    #     conv_history_tokens = num_tokens_from_messages(conversation)

    response = openai.ChatCompletion.create(
        engine=deployment_name, # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
        messages=conversation,
        temperature=0.7,
        max_tokens=max_response_tokens,
    )

    # print(response['choices'][0]['message']['content'])

    conversation.append({"role": "assistant", "content": response['choices'][0]['message']['content']})
    #print(conversation)

    responses=prompt_output_parse(response['choices'][0]['message']['content'])

    print(responses)

    # final_res.append(responses)

    # if not responses:
    #     #return empty
    #     return None


    return responses
        
        
            





In [29]:
print(final_res)

defaultdict(<class 'list'>, {'Hair Care': ['Haircare', 'Hair treatment', 'Hair products', 'Hair maintenance', 'Hair solutions'], 'Shaving': ['Grooming', 'Trimming', 'Shorn', 'Clipping', 'Razoring'], 'Motorcycles': ['Bikes', 'Choppers', 'Cycles', 'Hogs', 'wheelers'], "Women's Formal Wear": ["Women's Dressy Attire", "Ladies' Evening Wear", 'Female Formal Clothing', "Women's Elegant Outfits", "Ladies' Dressier Clothes"], "Women's Clothing": ['Ladies fashion', 'Female attire', "Women's apparel", "Girls' outfits", 'Feminine wear'], 'Retail Industry': ['Commerce', 'Sales', 'Shops', 'Market', 'Trade'], 'Comedy Events': ['Comedy Shows', 'Funny Performances', 'Humorous Gatherings', 'Amusing Occasions', 'Jolly Events'], 'Household Supplies': ['Home goods', 'Housewares', 'Domestic essentials', 'Cleaning supplies', 'Daily necessities'], 'Home Security': ['House Safety', 'Residency Protection', 'Domicile Security', 'Property Guard', 'Homestead Defense'], 'Polar Travel': ['Arctic Tours', 'Ice Expedi

In [30]:


dep_name=deployment_stuff()
# print(dep_name)

# print(series_title[0])
# print(episode_title[0])
final_res=defaultdict(list)
for i in tqdm(range(len(Tiers))):
    try:
        res=call(dep_name,Tiers[i])
        final_res[Tiers[i]]=res
        sleep(5)
    except Exception as e:
        print(e)

with open("alternate_names.pkl", 'wb') as f:
    pkl.dump(final_res,f)
    # count=0
    # for series_title,episode_title,medium_synopsis in tqdm(zip(series_titles,episode_titles,medium_synopsiss)):
    #     if count==100:
    #             break
    #     try:
            
    #         final_res=call(dep_name,series_title,episode_title,medium_synopsis,Tier1,initial_dic)
    #         if not final_res:
    #             continue
    #         for idx in range(len(final_res)):
    #             for k in final_res[idx].keys():
    #                 iab_label.append(k)
    #                 relevance_clue.append(final_res[idx][k]['Description'])
    #                 relevance_score.append(final_res[idx][k]['Relevance Score'])
    #                 series.append(series_title)   
    #                 episode.append(episode_title)
    #                 synopsis.append(medium_synopsis)
    #         count+=1        
    #     except Exception as e:
    #         print(e)
    # #         continue

    # final_dic={"Series Title":series, "Episode Title": episode, "RT Synopsis": synopsis, "Predicted IAB Label":  iab_label, "Relevance Clue": relevance_clue, "Relevance Score": relevance_score}
    # df_result=pd.DataFrame.from_dict(final_dic)

    # df_result.to_csv("GPT_Predictions.csv", index=False)
    # df_result.to_excel("GPT_Predictions.xlsx", index=False)

    

    
    

    

In [31]:
final_res

defaultdict(list,
            {'Hair Care': ['Haircare',
              'Hair treatment',
              'Hair products',
              'Hair maintenance',
              'Hair solutions'],
             'Shaving': ['Grooming',
              'Trimming',
              'Shorn',
              'Clipping',
              'Razoring'],
             'Motorcycles': ['Bikes',
              'Choppers',
              'Cycles',
              'Hogs',
              'wheelers'],
             "Women's Formal Wear": ["Women's Dressy Attire",
              "Ladies' Evening Wear",
              'Female Formal Clothing',
              "Women's Elegant Outfits",
              "Ladies' Dressier Clothes"],
             "Women's Clothing": ['Ladies fashion',
              'Female attire',
              "Women's apparel",
              "Girls' outfits",
              'Feminine wear'],
             'Retail Industry': ['Commerce',
              'Sales',
              'Shops',
              'Market',
              'Tra